# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 2 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/NotAveragePooled/split_features_2s_all.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[2.70414866e-03, 2.79165306e-04, 2.78151976e-06, ...,
          5.40252245e-16, 1.56153176e-05, 1.55124408e-04],
         [2.85087147e-03, 2.95306839e-04, 3.42857310e-06, ...,
          1.44704444e-15, 1.58973962e-05, 1.57929313e-04],
         [3.04498922e-03, 3.15533762e-04, 4.71718391e-06, ...,
          3.38392743e-15, 1.63767277e-05, 1.62701248e-04],
         ...,
         [4.71950382e-06, 6.96979924e-07, 9.83989841e-06, ...,
          4.08672234e-08, 1.99395944e-05, 1.89381415e-04],
         [4.43196142e-06, 4.70733005e-07, 1.06044568e-07, ...,
          4.94281655e-10, 1.80252409e-05, 1.73290591e-04],
         [4.34125244e-06, 4.54310514e-07, 3.83112779e-09, ...,
          2.21867693e-13, 1.70996799e-05, 1.64420217e-04]],
 
        [[2.21674682e-07, 8.28132378e-09, 1.14113669e-16, ...,
          5.64442031e-13, 4.15756946e-08, 3.75276195e-07],
         [2.63981337e-07, 9.74223593e-09, 2.37362292e-16, ...,
          5.92175932e-13, 4.42072655e-08, 3.9971

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(22631,)

array([16, 18, 16,  8,  0,  7,  1, 13,  5,  0,  0, 10,  0,  3,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(22631, 60, 87)

array([[2.51750206e-04, 4.63635016e-05, 3.09758646e-12, ...,
        5.68676796e-13, 3.82335129e-08, 5.15132790e-07],
       [2.60138366e-04, 4.79124536e-05, 4.27148327e-12, ...,
        8.23245387e-13, 1.38072087e-07, 1.71942709e-06],
       [2.74948944e-04, 5.06501715e-05, 1.28938086e-11, ...,
        4.50645997e-12, 3.23376246e-07, 3.95625341e-06],
       ...,
       [1.43458462e-06, 2.64033143e-07, 9.53330979e-09, ...,
        1.02847336e-08, 3.21130295e-08, 2.44415090e-07],
       [1.34675330e-06, 2.40034602e-07, 7.10412014e-11, ...,
        6.88556118e-11, 1.90400122e-08, 2.32998140e-07],
       [1.29980401e-06, 2.31511356e-07, 3.06543286e-14, ...,
        6.60463375e-14, 1.98763233e-08, 2.44531143e-07]])

'mfcc'

(22631, 20, 87)

...

(22631, 12, 87)

array([[0.39349289, 1.        , 0.76565271, ..., 0.42511843, 0.53971734,
        0.87640076],
       [0.40241583, 0.3937928 , 0.25525414, ..., 1.        , 1.        ,
        0.97018468],
       [1.        , 0.50390339, 0.2457685 , ..., 0.80265059, 0.70749506,
        0.72226342],
       ...,
       [0.48373222, 0.35887895, 0.33238306, ..., 0.44334805, 0.30104765,
        0.54676064],
       [0.48656566, 0.43102517, 0.68563702, ..., 0.27053293, 0.23075898,
        0.29046017],
       [0.45659793, 0.77367048, 0.74913608, ..., 0.20811069, 0.18596628,
        0.23210441]])

'rms'

(22631, 1, 87)

array([[0.01566686, 0.01963366, 0.02267696, 0.02198208, 0.0236679 ,
        0.02365906, 0.02739311, 0.02892642, 0.02962335, 0.03161302,
        0.02964298, 0.02855054, 0.02637647, 0.02320143, 0.02127909,
        0.02082221, 0.02107353, 0.02044727, 0.0215389 , 0.02171538,
        0.02103248, 0.02137822, 0.02056518, 0.02047369, 0.02067877,
        0.02130214, 0.02133721, 0.02149762, 0.02284854, 0.02213321,
        0.02208988, 0.02265538, 0.02229255, 0.02197747, 0.02131361,
        0.02166785, 0.02138098, 0.02257396, 0.02174746, 0.02019637,
        0.01945506, 0.01891561, 0.02042461, 0.02097673, 0.02098343,
        0.02131834, 0.0205403 , 0.01974241, 0.02029415, 0.02024212,
        0.02057395, 0.02136144, 0.02033739, 0.02035925, 0.02131695,
        0.02042016, 0.02099958, 0.0204634 , 0.01956393, 0.01958668,
        0.01869692, 0.01864281, 0.01893451, 0.01934329, 0.02076074,
        0.019988  , 0.02005406, 0.01977013, 0.01903823, 0.02036951,
        0.01973328, 0.01997821, 0.01980979, 0.01

In [13]:
display(val_y.shape)
display(val_y[:15])

(6044,)

array([10,  2, 12,  0,  1, 18,  2,  2, 17,  2,  2,  2,  0, 17,  7],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(6044, 60, 87)

array([[2.17200396e-04, 4.53335971e-05, 1.29495222e-13, ...,
        1.21034555e-12, 5.17416951e-05, 4.10931854e-04],
       [2.20289099e-04, 4.59743197e-05, 1.15376292e-13, ...,
        2.42292258e-12, 5.26615844e-05, 4.18141088e-04],
       [2.25604917e-04, 4.70761660e-05, 1.25535822e-12, ...,
        5.34527052e-12, 5.42772074e-05, 4.30814610e-04],
       ...,
       [8.90270095e-05, 2.79290157e-05, 9.93993419e-06, ...,
        5.31224018e-06, 8.29869500e-06, 4.28595645e-05],
       [6.43069815e-05, 1.35614011e-05, 7.30470999e-08, ...,
        5.46549170e-08, 3.75281510e-06, 3.03586412e-05],
       [5.72050771e-05, 1.19834535e-05, 5.41687084e-11, ...,
        3.40280461e-11, 3.30304761e-06, 2.68653043e-05]])

'mfcc'

(6044, 20, 87)

...

(6044, 12, 87)

array([[0.6451998 , 0.64002737, 0.68638929, ..., 0.75993676, 0.63950148,
        0.55819746],
       [0.62599534, 0.62821159, 0.83698962, ..., 0.60072089, 0.6233961 ,
        0.60917188],
       [1.        , 1.        , 1.        , ..., 0.38736675, 0.41141488,
        0.28301741],
       ...,
       [0.296773  , 0.46498901, 0.57427021, ..., 0.64805023, 0.95710992,
        1.        ],
       [0.27340847, 0.37722187, 0.48655279, ..., 0.466577  , 0.80984084,
        0.76830987],
       [0.53433068, 0.48248988, 0.52454861, ..., 0.40279458, 0.54774282,
        0.67895615]])

'rms'

(6044, 1, 87)

array([[0.00496129, 0.0059438 , 0.00697781, 0.00701446, 0.00727218,
        0.00736816, 0.00718243, 0.00729699, 0.0080523 , 0.00894135,
        0.00983361, 0.01031962, 0.01031073, 0.01027326, 0.01062839,
        0.01090027, 0.01210153, 0.01223124, 0.01218716, 0.01219549,
        0.01137712, 0.01163485, 0.01261372, 0.01224262, 0.01199135,
        0.01139301, 0.00976132, 0.01036333, 0.0136469 , 0.0338731 ,
        0.06918351, 0.09940138, 0.13557129, 0.15677701, 0.16202463,
        0.1640733 , 0.16449983, 0.19005741, 0.2260389 , 0.23742491,
        0.22443756, 0.20471627, 0.15242663, 0.11055657, 0.09755287,
        0.04201634, 0.01273967, 0.0110363 , 0.01018296, 0.00948956,
        0.00891118, 0.00858342, 0.0082546 , 0.00820571, 0.00877933,
        0.00884571, 0.00906115, 0.0090993 , 0.00816916, 0.00799693,
        0.00757182, 0.00754186, 0.00973617, 0.01017493, 0.01046399,
        0.0105744 , 0.01057907, 0.01014374, 0.00990268, 0.00985158,
        0.00963962, 0.01019118, 0.01032453, 0.01

## **Random Forest Model**

In [15]:
train_results = {}
val_results = {}

### **With all the features**

In [16]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 8091)

In [17]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 8091)

Fit the model with training data

In [18]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [19]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3881535407015222


In [20]:
train_yhat_result = rf.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3120
           1       1.00      1.00      1.00      2276
           2       1.00      1.00      1.00      1559
           3       1.00      1.00      1.00      1459
           4       1.00      1.00      1.00       223
           5       1.00      1.00      1.00       362
           6       1.00      1.00      1.00       434
           7       1.00      1.00      1.00      1295
           8       1.00      1.00      1.00       381
           9       1.00      1.00      1.00       658
          10       1.00      1.00      1.00       904
          11       1.00      1.00      1.00       349
          12       1.00      1.00      1.00      2234
          13       1.00      1.00      1.00       621
          14       1.00      1.00      1.00       891
          15       1.00      1.00      1.00      1310
          16       1.00      1.00      1.00     

In [21]:
val_yhat_result = rf.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.71      0.44       834
           1       0.28      0.49      0.36       384
           2       0.84      0.34      0.48      1441
           3       0.51      0.83      0.63       307
           4       0.07      0.01      0.02       154
           5       0.08      0.08      0.08        64
           6       0.09      0.02      0.03        65
           7       0.56      0.40      0.47       354
           8       0.08      0.02      0.03       127
           9       0.80      0.59      0.68       210
          10       0.35      0.31      0.33       243
          11       0.00      0.00      0.00       122
          12       0.31      0.60      0.41       287
          13       0.00      0.00      0.00       114
          14       0.03      0.08      0.04        65
          15       0.48      0.32      0.38       438
          16       0.29      0.28      0.29   

### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 1740)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 1740)

In [24]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.37177365982792854


### **With Mel-Spectrogram Only**

In [26]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 5220)

In [27]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 5220)

In [28]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [29]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3307412309728657


## **With Chroma Only**

In [30]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 1044)

In [31]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 1044)

In [32]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [33]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.20913302448709464


## **With MFCCs and Mel-Spectrogram**

In [34]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 6960)

In [35]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 6960)

In [36]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [37]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.39626075446724024


## **With MFCCs and Chroma**

In [38]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 2784)

In [39]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 2784)

In [40]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [41]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.37326273990734615


## **With MFCCs and RMS**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 1827)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 1827)

In [44]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3813699536730642


## **With Chroma and RMS**

In [46]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 1131)

In [47]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 1131)

In [48]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [49]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2938451356717406


## **With Mel-Spectrogram and Chroma**

In [50]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 6264)

In [51]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 6264)

In [52]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [53]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3259430840502978


## **With Mel-Spectrogram and RMS**

In [54]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 5307)

In [55]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 5307)

In [56]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [57]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.36052283256121775


## **With Mel-Spectrogram, MFCCs, RMS**

In [58]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 7047)

In [59]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 7047)

In [60]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [61]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3944407677035076


## **With Mel-Spectrogram, MFCCs, Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(22631, 8004)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(6044, 8004)

In [64]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3916280608868299


# Review results from all models

In [66]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.39
3,chroma,1.0,0.21
7,chroma_rms,1.0,0.29
2,melspectrogram,1.0,0.33
8,melspectrogram_chroma,1.0,0.33
11,melspectrogram_chroma_mfcc,1.0,0.39
9,melspectrogram_rms,1.0,0.36
10,melspectrogram_rms_mfcc,1.0,0.39
1,mfcc,1.0,0.37
5,mfcc_chroma,1.0,0.37
